# Exercise: Nonlinear Programming¶

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from casadi import (
    MX, Function, 
    vertcat, horzcat,
    gradient, hessian, jacobian,
    tanh, sin, cos
)


In [2]:
plot_dir = 'plots'
if not os.path.exists(plot_dir):
    os.mkdir(plot_dir)

## Sequential Quadratic Programming (SQP) Method

### 1. Decision variables and expressions for f, g, h

In [3]:
x = MX.sym('x', 2)
x.shape

(2, 1)

In [4]:
f = x[0]**2 + tanh(x[1]) ** 2
f

MX((sq(x[0])+sq(tanh(x[1]))))

In [5]:
g = cos(x[0] + x[1]) + 0.5
g

MX((0.5+cos((x[0]+x[1]))))

In [6]:
h = sin(x[0]) + 0.5
h

MX((0.5+sin(x[0])))

### 2. Symbolic expression for the Lagrangian

In [7]:
lam = MX.sym('λ')
nu = MX.sym('ν')
lagrangian_exp = f + lam * g + nu * h
lagrangian_exp

MX((((sq(x[0])+sq(tanh(x[1])))+(λ*(0.5+cos((x[0]+x[1])))))+(ν*(0.5+sin(x[0])))))

In [8]:
lagrangian_func = Function('lag', [x, lam, nu], [lagrangian_exp])
lagrangian_func

Function(lag:(i0[2],i1,i2)->(o0) MXFunction)

In [9]:
lag = lagrangian_func(vertcat(-0.5, -1.8), 2, 3)
lag

DM(0.875613)

In [10]:
assert abs(lag - 0.875613) < 1e-6

### 3. QP approximation setup

In [11]:
df_dx = jacobian(f, x)
df_dx.shape

(1, 2)

In [12]:
d2lag_dx2 = hessian(lagrangian_exp, x)[0]
d2lag_dx2.shape

(2, 2)

In [13]:
dx = MX.sym('Δx', 2)
dx.shape

(2, 1)

In [14]:
dg_dx = jacobian(g, x)
dg_dx.shape

(1, 2)

In [15]:
dh_dx = jacobian(h, x)
dh_dx.shape

(1, 2)

In [25]:
formulate_QP_problem = Function(
    'QP_problem', 
    [x, lam, nu], 
    [f, g, h, df_dx, dg_dx, dh_dx, d2lag_dx2],
    ['x', 'lam', 'nu'], 
    ['f', 'g', 'h', 'df_dx', 'dg_dx', 'dh_dx', 'd2lag_dx2']
)
formulate_QP_problem

Function(QP_problem:(x[2],lam,nu)->(f,g,h,df_dx[1x2],dg_dx[1x2],dh_dx[1x2,1nz],d2lag_dx2[2x2]) MXFunction)

### 4. Evaluate numerically at a point

In [26]:
x0 = vertcat(-0.5, -1.8)
lam0 = 0.
nu0 = 0.
qp_prob = formulate_QP_problem(x=x0, lam=lam0, nu=nu0)
qp_prob

{'d2lag_dx2': DM(
 [[2, 0], 
  [0, -0.349887]]),
 'df_dx': DM([[-1, -0.196099]]),
 'dg_dx': DM([[0.745705, 0.745705]]),
 'dh_dx': DM([[0.877583, 00]]),
 'f': DM(1.14644),
 'g': DM(-0.166276),
 'h': DM(0.0205745)}

In [27]:
assert qp_prob['f'].shape == (1, 1)
assert qp_prob['g'].shape == (1, 1)
assert qp_prob['h'].shape == (1, 1)
assert qp_prob['df_dx'].shape == (1, 2)
assert qp_prob['dg_dx'].shape == (1, 2)
assert qp_prob['dh_dx'].shape == (1, 2)
assert qp_prob['d2lag_dx2'].shape == (2, 2)

In [28]:
qp_prob['d2lag_dx2']

DM(
[[2, 0], 
 [0, -0.349887]])

In [32]:
assert np.max(np.abs(
    qp_prob['d2lag_dx2'] - np.array([[2, 0], [0, -0.3499]])
)) < 1e-4

### 5. Solve QP